In [1]:
from sympy import *
from IPython.display import Math, display

z, t = symbols('z t')

E = Function('E')(z,t) 
H = Function('H')(z,t)
D = Function('D')(z,t)


omega, k = symbols('omega k')

eps_r = Function('\\epsilon_r')(t)
eps_0, mu_0 = symbols('\\epsilon_0 \\mu_0')
omega = Symbol('\\omega')

MF = Derivative(E, z) + mu_0*Derivative(H, t)
MA = -Derivative(H, z) - eps_0*Derivative(eps_r*E, t)

display(Math(latex(MF)))
display(Math(latex(MA)))

EQ = Derivative(MF, z).doit()
display(Math(latex(EQ)))

EQ = EQ.subs(Derivative(H, z), -eps_0*Derivative(eps_r*E, t)).simplify()
display(Math(latex(EQ)))

EQ = EQ.subs(E, D/eps_r)
display(Math(latex(EQ)))

D_0 = Function('D_0')(t)
n = Function('n')(t)
c = Symbol('c')

EQ = EQ.subs([(D, D_0*exp(I*k*z)), (eps_r, n**2)])/exp(I*k*z)
EQ = EQ.doit().simplify().subs(eps_0*mu_0, 1/c**2)
EQ = (-EQ*c**2).simplify()
display(Math(latex(EQ)))



In [2]:
eps_r0, eps_r1 = symbols('\\epsilon_{r0} \\epsilon_{r1}')
n_0, n_1 = symbols('n_0 n_1')
omega_0, omega_1 = symbols('\\omega_0 \\omega_1')
D_I, D_R, D_T = symbols('D_I D_R D_T')

EQ_I = EQ.subs([(D_0, D_I*exp(I*omega_0*t)), (n, n_0)]).doit()
EQ_R = EQ.subs([(D_0, D_R*exp(I*omega_1*t)), (n, n_1)]).doit()
EQ_T = EQ.subs([(D_0, D_T*exp(I*omega_1*t)), (n, n_1)]).doit()

display(Math(latex(EQ_I)))
display(Math(latex(EQ_R)))
display(Math(latex(EQ_T)))



In [14]:

EQ_0 = D_I*exp(I*omega_0*t-I*k*z)
EQ_1 = D_T*exp(I*omega_1*t-I*k*z) + D_R*exp(-I*omega_1*t-I*k*z)

EQ_0 = EQ_0.subs(omega_0, k*c/n_0)
EQ_1 = EQ_1.subs(omega_1, k*c/n_1)

EQs_t0 = (EQ_1 - EQ_0).subs(t, 0)/exp(-I*k*z)
dEQs_t0 = (Derivative(EQ_1, t) - Derivative(EQ_0, t)).subs(t, 0).doit()/exp(-I*k*z)

EQs_t0 = EQs_t0.simplify()
dEQs_t0 = dEQs_t0.simplify()

display(Math(latex(EQs_t0)))
display(Math(latex(dEQs_t0)))

D_TR_sub =  solve([EQs_t0, dEQs_t0], [D_T, D_R])
display(Math(latex(D_TR_sub)))

R = D_TR_sub[D_R]/D_I
T = D_TR_sub[D_T]/D_I

display(Math('R = ' + latex(R)))
display(Math('T = ' + latex(T)))

## E: poynting power (relations not clear yet)
##TODO: the energy conservation law to be checked according to the Poynting theorem
E_I = D_I**2/(n_0**3)
E_R = (R**2*D_I**2/n_0**3).simplify()
E_T = (T**2*D_I**2/n_1**3).simplify()

display(Math('E_I = ' + latex(E_I)))
display(Math('E_R = ' + latex(E_R)))
display(Math('E_T = ' + latex(E_T)))

E_TR = (E_T + E_R).simplify()
display(Math('E_T + E_R = ' + latex(E_TR)))

E_TR_E_I = (E_TR/E_I).simplify()
display(Math('(E_T + E_R) / E_I = ' + latex(E_TR_E_I)))


In [1]:
# R = (1 - n1/n0)/2 = (n0 - n1)/2n0
# T = (1 + n1/n0)/2 = (n0 + n1)/2n0
# T + R = 1
# T**2 - R**2 = n1/n0

import numpy as np
from scipy import constants
import math, cmath

def GetRT(n0, n1):
    R = (n0-n1)/(2.0*n0)
    T = (n0+n1)/(2.0*n0)
    return [R, T]

def propWavesForward(waves, n0, k, t):
    for i in range(len(waves)):
        omega = k*constants.c/n0
        waves[i] = waves[i]*cmath.exp(1j*omega*t)
        
def propWavesBackward(waves, n0, k, t):
    for i in range(len(waves)):
        omega = k*constants.c/n0
        waves[i] = waves[i]*cmath.exp(-1j*omega*t)

def switchMedium(waves_F, waves_B, n0, n1):
    NF = len(waves_F)
    NB = len(waves_B)
    for i in range(NF):
        R, T = GetRT(n0, n1)
        waves_F.append(T*waves_F[i])
        waves_B.append(R*waves_F[i])
    for i in range(NB):
        R, T = GetRT(n0, n1)
        waves_F.append(R*waves_B[i])
        waves_B.append(T*waves_B[i])

waves_F = [1.0]
waves_B = []

N = 5
omega = 2.0*math.pi*constants.giga
k = omega/constants.c
n0, n1 = 1.0, 1.5
dt = 1.0*constants.nano
for i in range(N):
    switchMedium(waves_F, waves_B, n0, n1)
    propWavesForward(waves_F, n1, k, dt)
    propWavesBackward(waves_B, n1, k, dt)
    print(len(waves_F), len(waves_B))
    
    switchMedium(waves_F, waves_B, n1, n0)
    propWavesForward(waves_F, n0, k, dt)
    propWavesBackward(waves_B, n0, k, dt)
    print(len(waves_F), len(waves_B))
    
print(len(waves_F))



2 1
5 4
14 13
41 40
122 121
365 364
1094 1093
3281 3280
9842 9841
29525 29524
29525


In [14]:
# R = (1 - n1/n0)/2 = (n0 - n1)/2n0
# T = (1 + n1/n0)/2 = (n0 + n1)/2n0
# T + R = 1
# T**2 - R**2 = n1/n0

%pylab

import numpy as np
from scipy import constants
import math, cmath
import time

def GetRT(n0, n1):
    R = (n0-n1)/(2.0*n0)
    T = (n0+n1)/(2.0*n0)
    return [R, T]

def propWavesForward(w, n0, k, t):
    omega = k*constants.c/n0
    return w*cmath.exp(1j*omega*t)
        
def propWavesBackward(w, n0, k, t):
    omega = k*constants.c/n0
    return w*cmath.exp(-1j*omega*t)

def switchMedium(w_F, w_B, n0, n1):
    R, T = GetRT(n0, n1)
    w_F_next = T*w_F
    w_B_next = R*w_F

    w_F_next += R*w_B
    w_B_next += T*w_B
    return [w_F_next, w_B_next]

def plotField(w_F, w_B, k, n, t_st, dt):
    omega = k*constants.c/n
    t = np.linspace(0.0, dt, n_t, endpoint=True)
    w_tot = w_F*np.exp(1j*omega*t) + w_B*np.exp(-1j*omega*t)
    w_tot_prop = None
    if abs(w_F)<abs(w_B):
        w_tot_prop = (w_B - conjugate(w_F))*np.exp(-1j*omega*t) #+ 2.0*abs(w_F)*np.cos(omega*t + angle(w_F))
    else:
        w_tot_prop = (w_F - conjugate(w_B))*np.exp(1j*omega*t) #+ 2.0*abs(w_B)*np.cos(omega*t + angle(conjugate(w_B)))
    
    #plot(t_st + t, np.real(w_tot), 'r')
    plot(t_st + t, np.real(w_tot_prop), 'r')
    draw()
    time.sleep(0.1)
    

w_F = 1.0
w_B = 0.0

N = 60
f = 1.0*constants.giga
omega = 2.0*math.pi*f
k = omega/constants.c
n0, n1 = 1.5, 1.9
dt = 0.5*constants.nano

n_t = 50

for i in range(N):
    #clf()
    t_st = 2.0*i*dt

    w_F, w_B = switchMedium(w_F, w_B, n0, n1)
    
    plotField(w_F, w_B, k, n1, t_st, dt)

    w_F = propWavesForward(w_F, n1, k, dt)
    w_B = propWavesBackward(w_B, n1, k, dt)
    #print(w_F, w_B)

    w_F, w_B = switchMedium(w_F, w_B, n1, n0)
    
    plotField(w_F, w_B, k, n0, t_st+dt, dt)
    
    w_F = propWavesForward(w_F, n0, k, dt)
    w_B = propWavesBackward(w_B, n0, k, dt)
    #print(w_F, w_B)


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


`%pylab --no-import-all` prevents importing * from pylab and numpy
